In [ ]:
# processData to C2C history
import requests
import hmac
import hashlib
import pandas as pd
import pytz
from datetime import datetime, timedelta
import time

def processData(apiKey, apiSecret, beginTime=None, endTime=None, timezone=None):
    function_startTime = time.time()
    # endpoint
    base_url = 'https://api.binance.com'
    endpoint = '/sapi/v1/c2c/orderMatch/listUserOrderHistory'

    # 设置请求headers
    headers = {
        'X-MBX-APIKEY': apiKey,
    } 

    # 创建一个空的 DataFrame，用于存放结果
    all_transfers = pd.DataFrame()
    
    # 以每30天为间隔进行循环
    while (1):
        time.sleep(1)
        if beginTime > endTime:
            break
        # 获得当前循环的结束时间
        currentEndTime = datetime.strptime(beginTime, "%Y-%m-%d") + timedelta(days=30)
        # 如果当前结束时间超过了指定的结束时间，则将其设置为指定的结束时间
        if currentEndTime > datetime.strptime(endTime, "%Y-%m-%d"):
            currentEndTime = datetime.strptime(endTime, "%Y-%m-%d")
        
        # 处理当前时间段的数据
       
        ### 处理日期和时区
        beginTime = beginTime + 'T00:00:00'
        currentEndTime = currentEndTime.strftime("%Y-%m-%d") + "T23:59:59"
        
        dt_begin = datetime.strptime(beginTime, "%Y-%m-%dT%H:%M:%S")
        dt_end = datetime.strptime(currentEndTime, "%Y-%m-%dT%H:%M:%S")

        # 获取目标时区的时区对象
        target_timezone = pytz.timezone(timezone)
        # 将日期时间对象转换为目标时区的时间
        localized_dt_begin = target_timezone.localize(dt_begin)
        localized_dt_end = target_timezone.localize(dt_end)
        # 将本地时间转换为 UTC 时间
        utc_begin = localized_dt_begin.astimezone(pytz.utc)
        utc_end = localized_dt_end.astimezone(pytz.utc)
        # 使用 strftime 函数将 datetime 对象格式化为指定格式的字符串
        beginTime = utc_begin.strftime('%Y-%m-%dT%H:%M:%SZ')
        EndTime = utc_end.strftime('%Y-%m-%dT%H:%M:%SZ')

        # 转化为时间戳
        start_time = int(datetime.strptime(beginTime, '%Y-%m-%dT%H:%M:%SZ').timestamp() * 1000)
        end_time = int(datetime.strptime(EndTime, '%Y-%m-%dT%H:%M:%SZ').timestamp() * 1000)
        
        ## buy数据
        print('开始获取Buy数据')
        # 构造请求参数
        params_buy = {
            'tradeType': 'BUY',
            'rows': 100,
            'startTimestamp': start_time,
            'endTimestamp': end_time,
            'recvWindow': 6000,
            'timestamp': int(time.time() * 1000)  # 以毫秒为单位的当前时间戳
        }

        # 计算签名
        query_string_buy = '&'.join([f"{k}={v}" for k, v in params_buy.items()])
        signature_buy = hmac.new(apiSecret.encode('utf-8'), query_string_buy.encode('utf-8'), hashlib.sha256).hexdigest()
        params_buy['signature'] = signature_buy
        page = 0

        try:
            response = requests.get(base_url + endpoint, headers=headers, params=params_buy)
            response.raise_for_status()  # 如果请求不成功，会抛出异常
            df = pd.DataFrame(response.json()['data'])
        except requests.exceptions.RequestException as e:
            print("Error:", e)
            break
            
        if df.empty: # 如果没有buy数据
            print("没有Buy数据")

        else:
            trade_num = len(df)
            df_new = df
        
            while trade_num == 100: # 数据条数超出限制
                page += 1
                time.sleep(1)
        
                params_in = {
                    'tradeType': 'BUY',
                    'rows': 100,
                    'startTimestamp': start_time,
                    'endTimestamp': end_time,
                    'recvWindow': 6000,
                    'page': page,
                    'timestamp': int(time.time() * 1000)  # 以毫秒为单位的当前时间戳
                }

                query_string_in = '&'.join([f"{k}={v}" for k, v in params_in.items()])
                signature_in = hmac.new(apiSecret.encode('utf-8'), query_string_in.encode('utf-8'), hashlib.sha256).hexdigest()
                params_in['signature'] = signature_in

                try:
                    response = requests.get(base_url + endpoint, headers=headers, params=params_in)
                    response.raise_for_status()
                    df_new = pd.DataFrame(response.json()['data'])
                    trade_num = len(df_new)
                    df = pd.concat([df, df_new], ignore_index=True)
            
                except requests.exceptions.RequestException as e:
                    print("Error:", e)
                    break
            
            print(f'共获取{len(df)}条Buy数据')

        ## Sell
        print('开始获取Sell数据')
        # 构造请求参数
        params_sell = {
            'tradeType': 'SELL',
            'rows': 100,
            'startTimestamp': start_time,
            'endTimestamp': end_time,
            'recvWindow': 6000,
            'timestamp': int(time.time() * 1000)  # 以毫秒为单位的当前时间戳
        }

        # 计算签名
        query_string_sell = '&'.join([f"{k}={v}" for k, v in params_sell.items()])
        signature_sell = hmac.new(apiSecret.encode('utf-8'), query_string_sell.encode('utf-8'), hashlib.sha256).hexdigest()
        params_sell['signature'] = signature_sell
        page = 0

        try:
            response = requests.get(base_url + endpoint, headers=headers, params=params_sell)
            response.raise_for_status()  # 如果请求不成功，会抛出异常
            df_sell = pd.DataFrame(response.json()['data'])
        except requests.exceptions.RequestException as e:
            print("Error:", e)
            
        if df_sell.empty: # 如果没有buy数据
            print("没有Sell数据")

        else:
            trade_num = len(df_sell)
            df_new = df_sell
        
            while trade_num == 100: # 数据条数超出限制
                page += 1
                time.sleep(1)
        
                params_in = {
                    'tradeType': 'SELL',
                    'rows': 100,
                    'recvWindow': 6000,
                    'startTimestamp': start_time,
                    'endTimestamp': end_time,
                    'page': page,
                    'timestamp': int(time.time() * 1000)  # 以毫秒为单位的当前时间戳
                }

                query_string_in = '&'.join([f"{k}={v}" for k, v in params_in.items()])
                signature_in = hmac.new(apiSecret.encode('utf-8'), query_string_in.encode('utf-8'), hashlib.sha256).hexdigest()
                params_in['signature'] = signature_in

                try:
                    response = requests.get(base_url + endpoint, headers=headers, params=params_in)
                    response.raise_for_status()
                    df_new = pd.DataFrame(response.json()['data'])
                    trade_num = len(df_new)
                    df_sell = pd.concat([df_sell, df_new], ignore_index=True)
            
                except requests.exceptions.RequestException as e:
                    print("Error:", e)
                    break
            
            print(f'共获取{len(df_sell)}条Sell数据')

        df = pd.concat([df, df_sell], ignore_index=True)
        current_transfers = df
        if current_transfers is not None:
            # 将当前时间段的结果添加到总结果中
            all_transfers = pd.concat([all_transfers, current_transfers], ignore_index=True)
        
        # 更新下一次循环的开始时间为当前循环的结束时间的下一天
        currentEndTime = datetime.strptime(currentEndTime, "%Y-%m-%dT%H:%M:%S")
        beginTime = (currentEndTime + timedelta(days=1)).strftime("%Y-%m-%d")

    # 格式修改
    df['createTime'] = pd.to_datetime(df['datetime'], unit='ms').dt.strftime('%Y-%m-%dT%H:%M:%SZ')
    '''
    df.rename(columns={'symbol': 'currency', 'id': 'txid', 'time': 'datetime'}, inplace=True)
    df['datetime'] = pd.to_datetime(df['datetime'], unit='ms').dt.strftime('%Y-%m-%dT%H:%M:%SZ')
    df['contactPlatformSlug'] = ''
    df['contactIdentity'] = ''
    df['currency'] = temp
    df["type"] = ''
    df["direction"] = ''
    df['amount'] = ''
    '''
    #df_all_data.reset_index(drop=True, inplace=True)
    function_endTime = time.time()
    duration = function_endTime - function_startTime
    print(f"函数运行时间：{duration} 秒")

    return df

#apiKey = 'cPRKGQV6QGTOwpQUTK6VhbungH5rTy6xKL4TZQgipr6oPqPAgtnE5gfGpI2BIu0K'
#apiSecret = 'LbYip4Polxsnv2pMGoRdt3QGLmvD43555XH2iuXdsDs9V5r0C7SL9CDhzWbk9d1l'
apiKey = '0C9cLHlMnhilAvHKRI2XnWuAuC3tbzavNMUzN34ePNgkxj1W0WzOJzE4P0gen1fZ'
apiSecret = 'QdKKJKUS0fILSuLN5akEy7m7DkQv49IWP1QsoLSVQH4dZVuegzVybcUz2fzQdbjl'
transfers = processData(apiKey, apiSecret, beginTime = '2023-12-01', endTime = '2023-12-31', timezone = 'Asia/Shanghai')